In [ ]:
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from sklearn.metrics import accuracy_score
from tensorflow import keras
from sklearn import metrics
import tensorflow as tf
import pandas as pd
import numpy as np
import plotting
import hls4ml
import shap
import sys
import os

np.random.seed(77)

import matplotlib.pyplot as plt
import mplhep
plt.style.use(mplhep.style.CMS)

In [ ]:
options = {
    'v'          : '9',
    'date'       : '2022_09_14',
    'inTagIdent' : '_lTauPtCut18',
    'inTagCalib' : '_lTauPtCut18_uEtacut1.5',
    'caloClNxM'  : '5x9',
    'sparsity'   : 0.5
}

In [ ]:
indir = '/data_CMS/cms/motta/Phase2L1T/'+options['date']+'_v'+options['v']

N = int(options['caloClNxM'].split('x')[0])
M = int(options['caloClNxM'].split('x')[1])

sparsityTag = str(options['sparsity']).split('.')[0]+'p'+str(options['sparsity']).split('.')[1]

In [ ]:
X1_id = np.load(indir+'/TauCNNIdentifier'+options['caloClNxM']+'Training'+options['inTagIdent']+'/X_CNN_'+options['caloClNxM']+'_forEvaluator.npz')['arr_0']
X2_id = np.load(indir+'/TauCNNIdentifier'+options['caloClNxM']+'Training'+options['inTagIdent']+'/X_Dense_'+options['caloClNxM']+'_forEvaluator.npz')['arr_0']
Y_id  = np.load(indir+'/TauCNNIdentifier'+options['caloClNxM']+'Training'+options['inTagIdent']+'/Y_'+options['caloClNxM']+'_forEvaluator.npz')['arr_0']

X1_id = np.float32(X1_id)
X2_id = np.float32(X2_id)
Y_id = np.float32(Y_id)

X1_id_reduced = X1_id[:3000]
X2_id_reduced = X2_id[:3000]
Y_id_reduced = Y_id[:3000]

In [ ]:
X1_cal = np.load(indir+'/TauCNNCalibrator'+options['caloClNxM']+'Training'+options['inTagCalib']+'/X_CNN_'+options['caloClNxM']+'_forEvaluator.npz')['arr_0']
X2_cal = np.load(indir+'/TauCNNCalibrator'+options['caloClNxM']+'Training'+options['inTagCalib']+'/X_Dense_'+options['caloClNxM']+'_forEvaluator.npz')['arr_0']
Y_cal  = np.load(indir+'/TauCNNCalibrator'+options['caloClNxM']+'Training'+options['inTagCalib']+'/Y_'+options['caloClNxM']+'_forEvaluator.npz')['arr_0']

X1_cal = np.float32(X1_cal)
X2_cal = np.float32(X2_cal)
Y_cal = np.float32(Y_cal)

X1_cal_reduced = X1_cal[:3000]
X2_cal_reduced = X2_cal[:3000]
Y_cal_reduced = Y_cal[:3000]

In [ ]:
CNNmodel = keras.models.load_model(indir+'/TauCNNIdentifier'+options['caloClNxM']+'Training'+options['inTagIdent']+'/CNNmodel', compile=False)
QCNNmodel = keras.models.load_model(indir+'/TauCNNIdentifier'+options['caloClNxM']+'Training'+options['inTagIdent']+'/QCNNmodel', compile=False)

id_DNNmodel = keras.models.load_model(indir+'/TauCNNIdentifier'+options['caloClNxM']+'Training'+options['inTagIdent']+'/DNNmodel', compile=False)
id_QDNNmodel = keras.models.load_model(indir+'/TauCNNIdentifier'+options['caloClNxM']+'Training'+options['inTagIdent']+'/QDNNmodel', compile=False)

cal_DNNmodel = keras.models.load_model(indir+'/TauCNNCalibrator'+options['caloClNxM']+'Training'+options['inTagCalib']+'/TauCNNCalibrator', compile=False)
cal_QDNNmodel = keras.models.load_model(indir+'/TauCNNCalibrator'+options['caloClNxM']+'Training'+options['inTagCalib']+'/TauCNNQCalibrator', compile=False)

# NON-QUANTIZED MODELS FIRST

## Create HLS model for the identification CNN and DNN - NON-QUANTIZED

In [ ]:
############################## Pass non-quantized CNN model through hls4ml ##############################

hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                  rounding_mode='AP_RND',
                                                                                  saturation_mode='AP_SAT')

# baseline model
CNN_hls_cfg = hls4ml.utils.config_from_keras_model(CNNmodel, granularity='name')
CNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,7>'
CNN_hls_cfg['Model']['ReuseFactor'] = 1
for Layer in CNN_hls_cfg['LayerName'].keys():
    CNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Latency'
    CNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
    CNN_hls_cfg['LayerName'][Layer]['Trace'] = True
    CNN_hls_cfg['LayerName'][Layer]['Precision'] = 'ap_fixed<16,7>'

CNN_hls_cfg['LayerName']['TowerClusterImage']['Precision'] = 'ap_fixed<16,9>'

# CNN_hls_cfg['LayerName']['CNNlayer1']['Precision']['weight'] = 'ap_fixed<7,0>'
# CNN_hls_cfg['LayerName']['CNNlayer1']['Precision']['bias'] = 'ap_fixed<0,0>'
# CNN_hls_cfg['LayerName']['CNNlayer1']['Precision']['result'] = 'ap_fixed<16,7>'

# CNN_hls_cfg['LayerName']['BN_CNNlayer1']['Precision']['weight'] = 'ap_fixed<6,0>'
# CNN_hls_cfg['LayerName']['BN_CNNlayer1']['Precision']['bias'] = 'ap_fixed<6,1>'
# CNN_hls_cfg['LayerName']['BN_CNNlayer1']['Precision']['result'] = 'ap_fixed<16,7>'

# CNN_hls_cfg['LayerName']['RELU_CNNlayer1']['Precision'] = 'ap_fixed<16,7>'
# CNN_hls_cfg['LayerName']['MP_CNNlayer1']['Precision'] = 'ap_fixed<16,7>'

# CNN_hls_cfg['LayerName']['CNNlayer2']['Precision']['weight'] = 'ap_fixed<8,2>'
# CNN_hls_cfg['LayerName']['CNNlayer2']['Precision']['bias'] = 'ap_fixed<0,0>'
# CNN_hls_cfg['LayerName']['CNNlayer2']['Precision']['result'] = 'ap_fixed<16,6>'

# CNN_hls_cfg['LayerName']['BN_CNNlayer2']['Precision']['weight'] = 'ap_fixed<4,2>'
# CNN_hls_cfg['LayerName']['BN_CNNlayer2']['Precision']['bias'] = 'ap_fixed<6,3>'
# CNN_hls_cfg['LayerName']['BN_CNNlayer2']['Precision']['result'] = 'ap_fixed<16,6>'

# CNN_hls_cfg['LayerName']['RELU_CNNlayer2']['Precision'] = 'ap_fixed<16,6>'

# CNN_hls_cfg['LayerName']['middleMan']['Precision'] = 'ap_fixed<16,6>'
CNN_hls_cfg['LayerName']['middleMan']['Strategy'] = 'Stable'
print(CNN_hls_cfg)

CNN_cfg = hls4ml.converters.create_config(backend='Vivado')
CNN_cfg['IOType']     = 'io_parallel' # Must set this if using CNNs!
CNN_cfg['HLSConfig']  = CNN_hls_cfg
CNN_cfg['KerasModel'] = CNNmodel
CNN_cfg['OutputDir']  = 'CNNmodel/'
CNN_cfg['Part']       = 'xcvu13p-fhgb2104-2L-e'
CNN_cfg['XilinxPart'] = 'xcvu13p-fhgb2104-2L-e'

CNNmodel_hls = hls4ml.converters.keras_to_hls(CNN_cfg)

hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
CNNmodel_hls.compile()

hls4ml.model.profiling.numerical(model=CNNmodel, hls_model=CNNmodel_hls, X=[X1_id_reduced,X2_id_reduced])
hls4ml.utils.plot_model(CNNmodel_hls, show_shapes=True, show_precision=True, to_file=None)

In [ ]:
############################## Pass non-quantized identification DNN model through hls4ml ##############################

hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                  rounding_mode='AP_RND',
                                                                                  saturation_mode='AP_SAT')

# baseline model
id_DNN_hls_cfg = hls4ml.utils.config_from_keras_model(id_DNNmodel, granularity='name')
id_DNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
id_DNN_hls_cfg['Model']['ReuseFactor'] = 1
for Layer in id_DNN_hls_cfg['LayerName'].keys():
    id_DNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Latency'
    id_DNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
    id_DNN_hls_cfg['LayerName'][Layer]['Trace'] = True
id_DNN_hls_cfg['LayerName']['DNNlayer1']['Precision']['weight'] = 'ap_fixed<6,1>'
id_DNN_hls_cfg['LayerName']['DNNlayer1']['Precision']['result'] = 'ap_fixed<16,6>'
id_DNN_hls_cfg['LayerName']['RELU_DNNlayer1']['Precision'] = 'ap_fixed<9,6>'

id_DNN_hls_cfg['LayerName']['DNNlayer2']['Precision']['weight'] = 'ap_fixed<6,1>'
id_DNN_hls_cfg['LayerName']['DNNlayer2']['Precision']['result'] = 'ap_fixed<16,5>'
id_DNN_hls_cfg['LayerName']['RELU_DNNlayer2']['Precision'] = 'ap_fixed<8,5>'

id_DNN_hls_cfg['LayerName']['DNNout']['Precision']['weight'] = 'ap_fixed<6,1>'
id_DNN_hls_cfg['LayerName']['DNNout']['Precision']['result'] = 'ap_fixed<16,5>'

id_DNN_hls_cfg['LayerName']['sigmoid_DNNout']['Precision'] = 'ap_fixed<8,1>'
id_DNN_hls_cfg['LayerName']['sigmoid_DNNout']['Strategy'] = 'Stable'

print(id_DNN_hls_cfg)

id_DNN_cfg = hls4ml.converters.create_config(backend='Vivado')
id_DNN_cfg['IOType']     = 'io_parallel'
id_DNN_cfg['HLSConfig']  = id_DNN_hls_cfg
id_DNN_cfg['KerasModel'] = id_DNNmodel
id_DNN_cfg['OutputDir']  = 'id_DNNmodel/'
id_DNN_cfg['Part']       = 'xcvu13p-fhgb2104-2L-e'
id_DNN_cfg['XilinxPart'] = 'xcvu13p-fhgb2104-2L-e'

id_DNNmodel_hls = hls4ml.converters.keras_to_hls(id_DNN_cfg)

hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
id_DNNmodel_hls.compile()

id_CNNoutput = np.array(CNNmodel([X1_id_reduced,X2_id_reduced]))

hls4ml.model.profiling.numerical(model=id_DNNmodel, hls_model=id_DNNmodel_hls, X=id_CNNoutput)
hls4ml.utils.plot_model(id_DNNmodel_hls, show_shapes=True, show_precision=True, to_file=None)

## Create HLS model for the calibration DNN - NON-QUANTIZED

In [ ]:
############################## Pass non-quantized calibration DNN model through hls4ml ##############################

hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                  rounding_mode='AP_RND',
                                                                                  saturation_mode='AP_SAT')

# baseline model
cal_DNN_hls_cfg = hls4ml.utils.config_from_keras_model(cal_DNNmodel, granularity='name')
cal_DNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
cal_DNN_hls_cfg['Model']['ReuseFactor'] = 1
for Layer in cal_DNN_hls_cfg['LayerName'].keys():
    cal_DNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Latency'
    cal_DNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
    cal_DNN_hls_cfg['LayerName'][Layer]['Trace'] = True
#     cal_DNN_hls_cfg['LayerName'][Layer]['Precision'] = 'ap_fixed<16,7>'

cal_DNN_hls_cfg['LayerName']['DNNlayer1']['Precision']['weight'] = 'ap_fixed<8,2>'
cal_DNN_hls_cfg['LayerName']['DNNlayer1']['Precision']['result'] = 'ap_fixed<16,7>'
cal_DNN_hls_cfg['LayerName']['RELU_DNNlayer1']['Precision'] = 'ap_fixed<16,7>'

cal_DNN_hls_cfg['LayerName']['DNNlayer2']['Precision']['weight'] = 'ap_fixed<6,2>'
cal_DNN_hls_cfg['LayerName']['DNNlayer2']['Precision']['result'] = 'ap_fixed<16,9>'
cal_DNN_hls_cfg['LayerName']['RELU_DNNlayer2']['Precision'] = 'ap_fixed<16,9>'

cal_DNN_hls_cfg['LayerName']['DNNout']['Precision']['weight'] = 'ap_fixed<6,1>'
cal_DNN_hls_cfg['LayerName']['DNNout']['Precision']['result'] = 'ap_fixed<15,10>'

id_DNN_hls_cfg['LayerName']['DNNout']['Strategy'] = 'Stable'
print(cal_DNN_hls_cfg)

cal_DNN_cfg = hls4ml.converters.create_config(backend='Vivado')
cal_DNN_cfg['IOType']     = 'io_parallel'
cal_DNN_cfg['HLSConfig']  = cal_DNN_hls_cfg
cal_DNN_cfg['KerasModel'] = cal_DNNmodel
cal_DNN_cfg['OutputDir']  = 'cal_DNNmodel/'
cal_DNN_cfg['Part']       = 'xcvu13p-fhgb2104-2L-e'
cal_DNN_cfg['XilinxPart'] = 'xcvu13p-fhgb2104-2L-e'

cal_DNNmodel_hls = hls4ml.converters.keras_to_hls(cal_DNN_cfg)

hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
cal_DNNmodel_hls.compile()

cal_CNNoutput = np.array(CNNmodel([X1_cal_reduced,X2_cal_reduced]))

hls4ml.model.profiling.numerical(model=cal_DNNmodel, hls_model=cal_DNNmodel_hls, X=cal_CNNoutput)
hls4ml.utils.plot_model(cal_DNNmodel_hls, show_shapes=True, show_precision=True, to_file=None)

# QUANTIZED MODELS THEN

## Create HLS model for the identification CNN and DNN - QUANTIZED

In [ ]:
############################## Pass quantized CNN model through hls4ml ##############################

hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                  rounding_mode='AP_RND',
                                                                                  saturation_mode='AP_SAT')

# baseline model
QCNN_hls_cfg = hls4ml.utils.config_from_keras_model(QCNNmodel, granularity='name')
QCNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
QCNN_hls_cfg['Model']['ReuseFactor'] = 1
for Layer in QCNN_hls_cfg['LayerName'].keys():
    QCNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Latency'
    QCNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
    QCNN_hls_cfg['LayerName'][Layer]['Trace'] = True
    QCNN_hls_cfg['LayerName'][Layer]['Precision'] = 'ap_fixed<16,7>'

QCNN_hls_cfg['LayerName']['TowerClusterImage']['Precision'] = 'ap_fixed<16,9>'

# QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['weight'] = 'ap_fixed<6,0>'
# QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['bias'] = 'ap_fixed<0,0>'
# QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['result'] = 'ap_fixed<16,7>'

# QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer1']['Precision'] = 'ap_fixed<16,7>'
# QCNN_hls_cfg['LayerName']['MP_CNNpBNlayer1']['Precision'] = 'ap_fixed<16,7>'

# QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['weight'] = 'ap_fixed<2,0>'
# QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['bias'] = 'ap_fixed<0,0>'
# QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['result'] = 'ap_fixed<16,6>'

# QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer2']['Precision'] = 'ap_fixed<16,6>'

# QCNN_hls_cfg['LayerName']['middleMan']['Precision'] = 'ap_fixed<16,6>'
QCNN_hls_cfg['LayerName']['middleMan']['Strategy'] = 'Stable'
print(QCNN_hls_cfg)

QCNN_cfg = hls4ml.converters.create_config(backend='Vivado')
QCNN_cfg['IOType']     = 'io_parallel' # Must set this if using CNNs!
QCNN_cfg['HLSConfig']  = QCNN_hls_cfg
QCNN_cfg['KerasModel'] = QCNNmodel
QCNN_cfg['OutputDir']  = 'QCNNmodel/'
QCNN_cfg['Part']       = 'xcvu13p-fhgb2104-2L-e'
QCNN_cfg['XilinxPart'] = 'xcvu13p-fhgb2104-2L-e'

QCNNmodel_hls = hls4ml.converters.keras_to_hls(QCNN_cfg)

hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
QCNNmodel_hls.compile()

hls4ml.model.profiling.numerical(model=QCNNmodel, hls_model=QCNNmodel_hls, X=[X1_id_reduced,X2_id_reduced])
hls4ml.utils.plot_model(QCNNmodel_hls, show_shapes=True, show_precision=True, to_file=None)

In [ ]:
############################## Pass quantized identification DNN model through hls4ml ##############################

hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                  rounding_mode='AP_RND',
                                                                                  saturation_mode='AP_SAT')

# baseline model
id_QDNN_hls_cfg = hls4ml.utils.config_from_keras_model(id_QDNNmodel, granularity='name')
id_QDNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
id_QDNN_hls_cfg['Model']['ReuseFactor'] = 1
for Layer in id_QDNN_hls_cfg['LayerName'].keys():
    id_QDNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Latency'
    id_QDNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
    id_QDNN_hls_cfg['LayerName'][Layer]['Trace'] = True
    id_QDNN_hls_cfg['LayerName'][Layer]['Precision'] = 'ap_fixed<16,7>'
    
# id_QDNN_hls_cfg['LayerName']['DNNlayer1']['Precision']['weight'] = 'ap_fixed<6,0>'
# id_QDNN_hls_cfg['LayerName']['DNNlayer1']['Precision']['result'] = 'ap_fixed<16,7>'
# id_QDNN_hls_cfg['LayerName']['RELU_DNNlayer1']['Precision'] = 'ap_fixed<9,6>'

# id_QDNN_hls_cfg['LayerName']['DNNlayer2']['Precision']['weight'] = 'ap_fixed<6,0>'
# id_QDNN_hls_cfg['LayerName']['DNNlayer2']['Precision']['result'] = 'ap_fixed<16,6>'
# id_QDNN_hls_cfg['LayerName']['RELU_DNNlayer2']['Precision'] = 'ap_fixed<8,5>'

# id_QDNN_hls_cfg['LayerName']['DNNout']['Precision']['weight'] = 'ap_fixed<6,0>'
# id_QDNN_hls_cfg['LayerName']['DNNout']['Precision']['result'] = 'ap_fixed<16,5>'

# id_QDNN_hls_cfg['LayerName']['sigmoid_DNNout']['Precision'] = 'ap_fixed<8,1>'
id_QDNN_hls_cfg['LayerName']['sigmoid_DNNout']['Strategy'] = 'Stable'

print(id_QDNN_hls_cfg)

id_QDNN_cfg = hls4ml.converters.create_config(backend='Vivado')
id_QDNN_cfg['IOType']     = 'io_parallel'
id_QDNN_cfg['HLSConfig']  = id_QDNN_hls_cfg
id_QDNN_cfg['KerasModel'] = id_QDNNmodel
id_QDNN_cfg['OutputDir']  = 'id_QDNNmodel/'
id_QDNN_cfg['Part']       = 'xcvu13p-fhgb2104-2L-e'
id_QDNN_cfg['XilinxPart'] = 'xcvu13p-fhgb2104-2L-e'

id_QDNNmodel_hls = hls4ml.converters.keras_to_hls(id_QDNN_cfg)

hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
id_QDNNmodel_hls.compile()

id_QCNNoutput = np.array(QCNNmodel([X1_id_reduced,X2_id_reduced]))

hls4ml.model.profiling.numerical(model=id_QDNNmodel, hls_model=id_QDNNmodel_hls, X=id_QCNNoutput)
hls4ml.utils.plot_model(id_QDNNmodel_hls, show_shapes=True, show_precision=True, to_file=None)

## Create HLS model for the calibration DNN - QUANTIZED

In [ ]:
############################## Pass quantized calibration DNN model through hls4ml ##############################

hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                  rounding_mode='AP_RND',
                                                                                  saturation_mode='AP_SAT')

# baseline model
cal_QDNN_hls_cfg = hls4ml.utils.config_from_keras_model(cal_QDNNmodel, granularity='name')
cal_QDNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
cal_QDNN_hls_cfg['Model']['ReuseFactor'] = 1
for Layer in cal_QDNN_hls_cfg['LayerName'].keys():
    cal_QDNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Latency'
    cal_QDNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
    cal_QDNN_hls_cfg['LayerName'][Layer]['Trace'] = True
cal_QDNN_hls_cfg['LayerName']['DNNlayer1']['Precision']['weight'] = 'ap_fixed<6,2>'
cal_QDNN_hls_cfg['LayerName']['DNNlayer1']['Precision']['result'] = 'ap_fixed<16,7>'
cal_QDNN_hls_cfg['LayerName']['reluDNNlayer1']['Precision'] = 'ap_fixed<16,7>'

cal_QDNN_hls_cfg['LayerName']['DNNlayer2']['Precision']['weight'] = 'ap_fixed<6,2>'
cal_QDNN_hls_cfg['LayerName']['DNNlayer2']['Precision']['result'] = 'ap_fixed<16,9>'
cal_QDNN_hls_cfg['LayerName']['reluDNNlayer2']['Precision'] = 'ap_fixed<16,9>'

cal_QDNN_hls_cfg['LayerName']['DNNout']['Precision']['weight'] = 'ap_fixed<6,1>'
cal_QDNN_hls_cfg['LayerName']['DNNout']['Precision']['result'] = 'ap_fixed<15,10>'
print(cal_QDNN_hls_cfg)

cal_QDNN_cfg = hls4ml.converters.create_config(backend='Vivado')
cal_QDNN_cfg['IOType']     = 'io_parallel'
cal_QDNN_cfg['HLSConfig']  = cal_QDNN_hls_cfg
cal_QDNN_cfg['KerasModel'] = cal_QDNNmodel
cal_QDNN_cfg['OutputDir']  = 'cal_QDNNmodel/'
cal_QDNN_cfg['Part']       = 'xcvu13p-fhgb2104-2L-e'
cal_QDNN_cfg['XilinxPart'] = 'xcvu13p-fhgb2104-2L-e'

cal_QDNNmodel_hls = hls4ml.converters.keras_to_hls(cal_QDNN_cfg)

hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
cal_QDNNmodel_hls.compile()

cal_QCNNoutput = np.array(QCNNmodel([X1_cal_reduced,X2_cal_reduced]))

hls4ml.model.profiling.numerical(model=cal_QDNNmodel, hls_model=cal_QDNNmodel_hls, X=cal_QCNNoutput)
hls4ml.utils.plot_model(cal_QDNNmodel_hls, show_shapes=True, show_precision=True, to_file=None)

# COMPARE HLS TO KERAS AND QKERAS MODELS

In [ ]:
def plotROC(Y, y_pred, y_pred_hls4ml, label="Model"):
    
    accuracy_keras  = float(accuracy_score(np.argmax(Y,axis=1), np.argmax(y_pred,axis=1), normalize=True))
    accuracy_hls4ml = float(accuracy_score(np.argmax(Y,axis=1), np.argmax(y_pred_hls4ml,axis=1), normalize=True))

    print("Accuracy Keras:  {}".format(accuracy_keras))
    print("Accuracy hls4ml: {}".format(accuracy_hls4ml))
    
    fig, ax = plt.subplots(figsize=(10, 10))

    FPR, TPR, THR = metrics.roc_curve(Y, y_pred)
    AUC = metrics.roc_auc_score(Y, y_pred)
    
    FPR_HLS, TPR_HLS, THR_HLS = metrics.roc_curve(Y, y_pred_hls4ml)
    AUC_HLS = metrics.roc_auc_score(Y, y_pred_hls4ml)
    
    plt.plot(TPR, FPR, label='Keras ROC, AUC = %.3f' % (AUC),   color='blue',lw=2)
    plt.plot(TPR_HLS, FPR_HLS, label='HLS ROC, AUC = %.3f' % (AUC_HLS), color='green',lw=2)
    
    plt.legend(loc = 'upper left', fontsize=16)
    plt.xlabel('Signal Efficiency')
    plt.ylabel('Background Efficiency')
    mplhep.cms.label('Phase-2 Simulation', data=True, rlabel='14 TeV, 200 PU')
    plt.figtext(0.2, 0.38,label, wrap=True, horizontalalignment='left',verticalalignment='center')
    plt.ylim(0.01,1.)
    plt.xlim(0.7,1.)

In [ ]:
# non-quantized models ID
y_ident        = id_DNNmodel.predict(CNNmodel.predict([X1_id, X2_id]))
y_ident_hls4ml = id_DNNmodel_hls.predict(CNNmodel_hls.predict([X1_id, X2_id]))

In [ ]:
# non-quantized models CALIB
y_calib        = cal_DNNmodel.predict(CNNmodel.predict([X1_cal, X2_cal]))
y_calib_hls4ml = cal_DNNmodel_hls.predict(CNNmodel_hls.predict([X1_cal, X2_cal]))

In [ ]:
# quantized models ID
y_ident_q        = id_QDNNmodel.predict(QCNNmodel.predict([X1_id, X2_id]))
y_ident_hls4ml_q = id_QDNNmodel_hls.predict(QCNNmodel_hls.predict([X1_id, X2_id]))

In [ ]:
# quantized models CALIB
y_calib_q        = cal_QDNNmodel.predict(QCNNmodel.predict([X1_cal, X2_cal]))
y_calib_hls4ml_q = cal_QDNNmodel_hls.predict(QCNNmodel_hls.predict([X1_cal, X2_cal]))

In [ ]:
# Plot the floating point model:    
plotROC(Y_id,y_ident,y_ident_hls4ml,label="Keras") 

# Plot the quantized QKeras model
plotROC(Y_id,y_ident_q,y_ident_hls4ml_q,label="QKeras") 

In [ ]:
ravelled = Y_cal[:,0]

response_base = y_calib.ravel() / ravelled
response_hls4ml = y_calib_hls4ml.ravel() / ravelled
response_q = y_calib_q.ravel() / ravelled
response__q_hls4ml = y_calib_hls4ml_q.ravel() / ravelled

In [ ]:
plt.figure(figsize=(10,10))
plt.hist(response_base,      bins=np.arange(0,5,0.1), label=r'Keras Calibrated response : $\mu$ = %.2f, $\sigma$ =  %.2f' % (np.mean(response_base), np.std(response_base)), color='blue', lw=2, density=True, histtype='step', alpha=0.7)
plt.hist(response_hls4ml,    bins=np.arange(0,5,0.1), label=r'HLS Calibrated response : $\mu$ = %.2f, $\sigma$ =  %.2f' % (np.mean(response_hls4ml), np.std(response_hls4ml)), color='green',lw=2, density=True, histtype='step', alpha=0.7)
plt.hist(response_q,         bins=np.arange(0,5,0.1), label=r'QKeras Calibrated response : $\mu$ = %.2f, $\sigma$ =  %.2f' % (np.mean(response_q), np.std(response_q)), color='blue',  ls='--', lw=2, density=True, histtype='step', alpha=0.7)
plt.hist(response__q_hls4ml, bins=np.arange(0,5,0.1), label=r'QHLS Calibrated response : $\mu$ = %.2f, $\sigma$ =  %.2f' % (np.mean(response__q_hls4ml), np.std(response__q_hls4ml)), color='green', ls='--', lw=2, density=True, histtype='step', alpha=0.7)
plt.xlabel(r'$p_{T}^{L1 \tau} / p_{T}^{Gen \tau}$')
plt.ylabel(r'a.u.')
# plt.xlim(0., 2.)
# plt.ylim(0., 3.5)
plt.legend(loc = 'upper right', fontsize=16)
plt.grid(linestyle='dotted')
mplhep.cms.label('Phase-2 Simulation', data=True, rlabel='14 TeV, 200 PU')